In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import cv2
import sklearn
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [2]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [3]:
def load_keypoint_sequences(data_path='gesture_recognition/Fall 2020/data_30', sequence_length=15):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        # print(gesture['label'], gesture['keypoints'])
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                gesture['keypoints'][i] = [[np.nan, np.nan, np.nan] for _ in range(21)]
                        for start_frame in range(len(gesture['keypoints']) - sequence_length + 1):
                            keypoints.append(gesture['keypoints'][start_frame: start_frame + sequence_length])
                            labels.append(LABEL_DICT[gesture['label']])
    keypoints = np.array(keypoints)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_keypoint_sequences()
print(keypoints.shape, labels.shape)

(60587, 15, 21, 3) (60587,)


In [4]:
def generate_connection_angles_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return np.arccos(angles)
angles = generate_connection_angles_from_sequences(keypoints)
print(angles.shape)

(60587, 15, 210)


In [5]:
def generate_joint_distances_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    distances = np.linalg.norm(tensor1-tensor2,axis=-1).transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return distances
distances = generate_joint_distances_from_sequences(keypoints)
print(distances.shape)

(60587, 15, 210)


In [6]:
def visualize_keypoint_sequences(keypoints):
    for sequence in keypoints:
        for points in sequence:
            img = np.zeros((480, 640, 3))
            for point in points:
                x, y, z = point
                if np.isnan(x):
                    continue
                cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
            for connection in CONNECTION_LABELS:
                if np.isnan(points[connection[0]][0]):
                    continue
                x0, y0, z0 = points[connection[0]]
                x1, y1, z1 = points[connection[1]]
                cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imshow("Key Points", img)
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                cv2.waitKey(1) # cannot close window on macOS without this line
                return
# visualize_keypoint_sequences(keypoints)

In [7]:
def process_sequence_features(keypoints, angles, distances):
    data_length = keypoints.shape[0]
    sequence_length = keypoints.shape[1]
    keypoints = keypoints.reshape(data_length*sequence_length, -1)
    angles = angles.reshape(data_length*sequence_length, -1)
    distances = distances.reshape(data_length*sequence_length, -1)
    features = np.concatenate((keypoints, angles, distances), -1)
    df = pd.DataFrame(features)
    df = (df-df.mean())/df.std()
    df = df.fillna(0)
    features = df.to_numpy().reshape(data_length, sequence_length, -1)
    return features
X = process_sequence_features(keypoints, angles, distances)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)
print(X.shape)

(60587, 15, 483)


## RNN Classification

In [9]:
model_lstm = tf.keras.Sequential([layers.Masking() ,layers.LSTM(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_lstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 48469 samples, validate on 12118 samples
Epoch 1/20
48469/48469 [==============================] - 71s 1ms/sample - loss: 0.5908 - accuracy: 0.8223 - val_loss: 0.6401 - val_accuracy: 0.8146
Epoch 2/20
48469/48469 [==============================] - 70s 1ms/sample - loss: 0.3040 - accuracy: 0.9047 - val_loss: 0.6520 - val_accuracy: 0.8204
Epoch 3/20
48469/48469 [==============================] - 71s 1ms/sample - loss: 0.2502 - accuracy: 0.9225 - val_loss: 0.6519 - val_accuracy: 0.8329
Epoch 4/20
48469/48469 [==============================] - 70s 1ms/sample - loss: 0.2150 - accuracy: 0.9352 - val_loss: 0.7150 - val_accuracy: 0.8263
Epoch 5/20
48469/48469 [==============================] - 70s 1ms/sample - loss: 0.1819 - accuracy: 0.9444 - val_loss: 0.7729 - val_accuracy: 0.8275
Epoch 6/20
48469/48469 [==============================] - 68s 1ms/sample - loss: 0.1677 - accuracy: 0.9473 - val_loss: 0.7251 - val_accuracy: 0.8316
Epoch 7/20
48469/48469 [==============================] 

In [8]:
model_gru = tf.keras.Sequential([layers.Masking() ,layers.GRU(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_gru.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru.fit(X, labels, epochs=20, validation_split=0.2)

Train on 48469 samples, validate on 12118 samples
Epoch 1/20
48469/48469 [==============================] - 92s 2ms/sample - loss: 0.4011 - accuracy: 0.8700 - val_loss: 0.5307 - val_accuracy: 0.8280
Epoch 2/20
48469/48469 [==============================] - 56s 1ms/sample - loss: 0.2318 - accuracy: 0.9246 - val_loss: 0.6315 - val_accuracy: 0.8179
Epoch 3/20
48469/48469 [==============================] - 55s 1ms/sample - loss: 0.1886 - accuracy: 0.9400 - val_loss: 0.6900 - val_accuracy: 0.8179
Epoch 4/20
48469/48469 [==============================] - 54s 1ms/sample - loss: 0.1624 - accuracy: 0.9485 - val_loss: 0.6912 - val_accuracy: 0.8237
Epoch 5/20
48469/48469 [==============================] - 55s 1ms/sample - loss: 0.1495 - accuracy: 0.9518 - val_loss: 0.7715 - val_accuracy: 0.8209
Epoch 6/20
48469/48469 [==============================] - 54s 1ms/sample - loss: 0.1271 - accuracy: 0.9599 - val_loss: 0.7342 - val_accuracy: 0.8215
Epoch 7/20
48469/48469 [==============================] 

GRU cells sometimes work not as well as LSTM cells, but is cheaper in theory.

In [10]:
model_bilstm = tf.keras.Sequential([layers.Masking(), tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(GESTURE_TYPES, activation=None)), tf.keras.layers.Activation('softmax')])
model_bilstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_bilstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 48469 samples, validate on 12118 samples
Epoch 1/20
48469/48469 [==============================] - 112s 2ms/sample - loss: 0.6122 - accuracy: 0.8259 - val_loss: 0.5871 - val_accuracy: 0.8194
Epoch 2/20
48469/48469 [==============================] - 102s 2ms/sample - loss: 0.3400 - accuracy: 0.8997 - val_loss: 0.6354 - val_accuracy: 0.8277
Epoch 3/20
48469/48469 [==============================] - 103s 2ms/sample - loss: 0.2925 - accuracy: 0.9128 - val_loss: 0.6672 - val_accuracy: 0.8220
Epoch 4/20
48469/48469 [==============================] - 87s 2ms/sample - loss: 0.2513 - accuracy: 0.9253 - val_loss: 0.7424 - val_accuracy: 0.8193
Epoch 5/20
48469/48469 [==============================] - 87s 2ms/sample - loss: 0.3849 - accuracy: 0.9087 - val_loss: 0.7436 - val_accuracy: 0.8193
Epoch 6/20
48469/48469 [==============================] - 86s 2ms/sample - loss: 0.2694 - accuracy: 0.9232 - val_loss: 0.7694 - val_accuracy: 0.8227
Epoch 7/20
48469/48469 [=============================

Making LSTM bidirectional does not improve the accuracy by a lot.

In [11]:
model_rnn = tf.keras.Sequential([layers.Masking() ,tf.keras.layers.SimpleRNN(GESTURE_TYPES, activation=None), tf.keras.layers.Activation('softmax')])
model_rnn.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_rnn.fit(X, labels, epochs=20, validation_split=0.2)

Train on 48469 samples, validate on 12118 samples
Epoch 1/20
48469/48469 [==============================] - 57s 1ms/sample - loss: 1.0035 - accuracy: 0.7062 - val_loss: 0.9609 - val_accuracy: 0.7347
Epoch 2/20
48469/48469 [==============================] - 49s 1ms/sample - loss: 0.5405 - accuracy: 0.8302 - val_loss: 0.9544 - val_accuracy: 0.7820
Epoch 3/20
48469/48469 [==============================] - 46s 954us/sample - loss: 0.4549 - accuracy: 0.8566 - val_loss: 0.8428 - val_accuracy: 0.7762
Epoch 4/20
48469/48469 [==============================] - 45s 919us/sample - loss: 0.4230 - accuracy: 0.8643 - val_loss: 0.8053 - val_accuracy: 0.7876
Epoch 5/20
48469/48469 [==============================] - 53s 1ms/sample - loss: 0.4059 - accuracy: 0.8686 - val_loss: 0.7734 - val_accuracy: 0.7965
Epoch 6/20
48469/48469 [==============================] - 46s 941us/sample - loss: 0.3876 - accuracy: 0.8754 - val_loss: 0.8141 - val_accuracy: 0.8040
Epoch 7/20
48469/48469 [==========================

Simple RNN gives a slightly worse performance.

In [12]:
model_lstm4 = tf.keras.Sequential([layers.Masking(), tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm4.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm4.fit(X, labels, epochs=20, validation_split=0.2)

Train on 48469 samples, validate on 12118 samples
Epoch 1/20
48469/48469 [==============================] - 144s 3ms/sample - loss: 0.2658 - accuracy: 0.9189 - val_loss: 0.5949 - val_accuracy: 0.8339
Epoch 2/20
48469/48469 [==============================] - 133s 3ms/sample - loss: 0.0836 - accuracy: 0.9768 - val_loss: 0.6417 - val_accuracy: 0.8395
Epoch 3/20
48469/48469 [==============================] - 146s 3ms/sample - loss: 0.0388 - accuracy: 0.9907 - val_loss: 0.7743 - val_accuracy: 0.8158
Epoch 4/20
48469/48469 [==============================] - 146s 3ms/sample - loss: 0.0339 - accuracy: 0.9912 - val_loss: 0.7350 - val_accuracy: 0.8359
Epoch 5/20
48469/48469 [==============================] - 147s 3ms/sample - loss: 0.0233 - accuracy: 0.9945 - val_loss: 0.7939 - val_accuracy: 0.8310
Epoch 6/20
48469/48469 [==============================] - 132s 3ms/sample - loss: 0.0236 - accuracy: 0.9935 - val_loss: 0.8854 - val_accuracy: 0.8304
Epoch 7/20
48469/48469 [==========================

Increasing depth may improve performance.

In [14]:
model_lstm5 = tf.keras.Sequential([layers.Masking(), tf.keras.layers.LSTM(128,return_sequences=True),tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm5.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm5.fit(X, labels, epochs=20, validation_split=0.2)

Train on 48469 samples, validate on 12118 samples
Epoch 1/20
48469/48469 [==============================] - 342s 7ms/sample - loss: 0.2515 - accuracy: 0.9207 - val_loss: 0.6492 - val_accuracy: 0.8213
Epoch 2/20
48469/48469 [==============================] - 295s 6ms/sample - loss: 0.0731 - accuracy: 0.9774 - val_loss: 0.7139 - val_accuracy: 0.8298
Epoch 3/20
48469/48469 [==============================] - 221s 5ms/sample - loss: 0.0447 - accuracy: 0.9869 - val_loss: 0.8252 - val_accuracy: 0.8293
Epoch 4/20
48469/48469 [==============================] - 214s 4ms/sample - loss: 0.0330 - accuracy: 0.9906 - val_loss: 0.8619 - val_accuracy: 0.8308
Epoch 5/20
48469/48469 [==============================] - 214s 4ms/sample - loss: 0.0274 - accuracy: 0.9921 - val_loss: 0.9212 - val_accuracy: 0.8183
Epoch 6/20
48469/48469 [==============================] - 212s 4ms/sample - loss: 0.0256 - accuracy: 0.9925 - val_loss: 0.8031 - val_accuracy: 0.8227
Epoch 7/20
48469/48469 [==========================